In [1]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmcv import Config
import copy
import os.path as osp
import os
import mmcv
import numpy as np
import time
import cv2
import json
from pylab import *
from skimage import measure 
from shapely.geometry import Polygon, MultiPolygon
import seaborn as sns
import matplotlib.pyplot as plt
import copy

In [2]:
epo = '30'

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
cfg = Config.fromfile('./configs/faster_rcnn/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou.py')
cfg.work_dir = 'tools/paper/train/logs/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou_Test2021_202103250741/'
cfg.gpu_ids = range(0,2)
cfg.model.inference = True
checkpoint = cfg.work_dir + 'epoch_' + epo + '.pth'
model = init_detector(cfg, checkpoint, device='cuda:0')
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

data_root = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/'

/root/workspace/Thyroid_Solid_Nodule/code/mmdetection/mmdet/apis/inference.py:41: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


/root/miniconda3/envs/mmdetection/lib/python3.6/site-packages/pycocotools/coco.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/annotations/pseudo_images.json' mode='r' encoding='UTF-8'>
  dataset = json.load(open(annotation_file, 'r'))


In [3]:
## 存mask文件和最后一层特征
def draw_features(x, savedir, savename):
    # print(savename)
    img = copy.deepcopy(x)
    output = copy.deepcopy(x)
    # 2022 img = np.where(img, 255, 0)
    img -= img.min()
    img /= (img.max() - img.min())
    img *=  256
#     img += 128
    img=img.astype(np.uint8)  #转成unit8
    img = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    cv2.imwrite(savedir + '/' + savename, img,[int(cv2.IMWRITE_JPEG_QUALITY),95])
#     print(img)
#     plt.figure(figsize = (10, 8))
#     ax = sns.heatmap(img, linewidths = 0.05, cmap='rainbow', annot = False)
    # sns.heatmap(img, linewidths = 0.05, ax = ax2, vmax=255, vmin=0, cmap='rainbow')
#     plt.savefig(savedir + '/' + savename)
#     plt.figure(figsize = img.shape)
#     plt.imshow(img, cmap='jet')
#     plt.axis('off')  
#     plt.savefig(savedir + '/' + savename, dpi=400)
#     plt.show()
    
    output = np.where(output >= 0.1, 255, 0)
    output=output.astype(np.uint8)
#     output = output.where(img, 255, 0)
    cv2.imwrite(savedir + '/out-' + savename,mmcv.bgr2rgb(output),[int(cv2.IMWRITE_JPEG_QUALITY),95])

In [4]:
def postprocess(image_dir, output_dir):
    for root, dirs, files in os.walk(os.path.abspath(image_dir)):
        for file in files:
            img = mmcv.imread(image_dir + file)
            result, x = inference_detector(model, img)
            img_np=x[0][0]
            draw_features(img_np, output_dir, file)

In [5]:
result_path = cfg.work_dir + '/test2022s/result/' + epo + '/'
if os.path.exists(result_path) == False:
    os.makedirs(result_path)
annotation_path = cfg.work_dir + '/test2022s/annotation/' + epo + '/'
if os.path.exists(annotation_path) == False:
    os.makedirs(annotation_path)
image_path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/tests/'
postprocess(image_path, result_path)
    # rebuild_json(result_path, data_root + 'annotations/tests.json', annotation_path, '12.json')
image_path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/validations/'
postprocess(image_path, result_path)
image_path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/demo2022/image/'
postprocess(image_path, result_path)

/root/workspace/Thyroid_Solid_Nodule/code/mmdetection/mmdet/core/post_processing/bbox_nms.py:52: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629416375/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  labels = valid_mask.nonzero()[:, 1]
